# Story 2 images

##### 1. find characters traits 
        1. use NER to find the main characters first appear 
        2. use SUMMARY to summary sentences between character +-$long 
        3. use VICUNA to generate prompts about characters' physical look

##### 2. highlights (!more about the verbs and the face emotions)
        1. find highligh sentence and check the character(s) who would on the picture
                may by coference resolution to check the pronun is who 
        2. find the verb or (face emotions)
        3. generate prompts

In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModel
from transformers import pipeline
import numpy as np
from fastcoref import FCoref
import re

In [2]:
f = open('story/story1.txt')
text = f.read()
f.close

story = text

In [3]:
story_lines = story.split('.')

story_lines

['\ufeffHi guys, my name is Tina',
 ' \nAll my life I have been shy and have struggled with self-esteem',
 ' \nThe kids at school teased me and were cruel to me',
 '\n[I was 14 when the most popular girl at school, Jessica, poured juice on my head',
 ']\nEveryone laughed as I cried in the bathroom, ashamed',
 '\nWhen I returned to the classroom, I saw a new student',
 ' \nShe approached me and told me her name was Mary',
 '\nWhen I told her what happened to me that day at school, Mary frowned',
 ' \nShe said I should take revenge',
 " \n[Although I had doubts, Mary put suspicious pills in Jessica's backpack the next day and called the police",
 ']\nJessica was in shock when two police officers found the pills in her bag',
 ' \nI was a little sorry for Jessica, but at the same time, I was happy that she realized what it was like to be humiliated',
 '\nI spent more and more time with Mary',
 ' \nWhile I was still shy, my best friend was wild and free-spirited',
 ' \nOne evening we saw a 

In [4]:

s = re.split('\[|\]', story) 
highlights = []
for i in range(len(s)):
    if i%2 == 1:
        highlights.append(s[i])

highlights

['I was 14 when the most popular girl at school, Jessica, poured juice on my head.',
 "Although I had doubts, Mary put suspicious pills in Jessica's backpack the next day and called the police.",
 'Supposedly, someone pushed him as he went down the steps to the subway.',
 'The school bell rang and suddenly all the kids in the school were standing in the hallway laughing at me.',
 'She opened her backpack, took out her books, and lit them with a match.',
 'Within a minute, the fire started spreading through the school.',
 'She told me that I created a character that was the complete opposite of me and that I called her Mary.']

In [5]:
matches = [(m.group(0), (m.start())) for m in re.finditer(r'\[|\]+', story)]
b, match_index = zip(*matches)
print(match_index)

(145, 226, 491, 598, 1527, 1599, 2190, 2296, 2494, 2566, 2568, 2632, 3348, 3452)


### Coference Resolution
To use the larger but more accurate LingMess model

In [6]:
model = FCoref()
preds = model.predict(
    texts=[story]
)

pronun_position = preds[0].get_clusters(as_strings=False)
pronun = preds[0].get_clusters()

/Users/sunny/mambaforge/envs/env_tf/lib/python3.9/site-packages/spacy/util.py:837: UserWarning: [W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.3.3). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
06/27/2023 00:17:55 - INFO - 	 missing_keys: []
06/27/2023 00:17:55 - INFO - 	 unexpected_keys: []
06/27/2023 00:17:55 - INFO - 	 mismatched_keys: []
06/27/2023 00:17:55 - INFO - 	 error_msgs: []
06/27/2023 00:17:55 - INFO - 	 Model Parameters: 90.5M, Transformer: 82.1M, Coref head: 8.4M
06/27/2023 00:17:55 - INFO - 	 Tokenize 1 inputs...


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

06/27/2023 00:17:55 - INFO - 	 ***** Running Inference on 1 texts *****


Inference:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:

for i in range(len(pronun)):
    print(pronun[i], pronun_position[i])

['my', 'my', 'I', 'me', 'me', 'I', 'my', 'I', 'I', 'I', 'me', 'me', 'I', 'me', 'I', 'I', 'I', 'I', 'I', 'I', 'my', 'I', 'I', 'I', 'I', 'my', 'I', 'I', 'I', 'me', 'me', 'I', 'I', 'me', 'my', 'I', 'me', 'I', 'I', 'me', 'I', 'I', 'my', 'I', 'me', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'me', 'I', 'I', 'I', 'I', 'me', 'me', 'me', 'I', 'I', 'I', 'I', 'me', 'my', 'I', 'I', 'myself', 'my', 'I', 'me', 'I', 'me', 'I', 'me', 'I', 'my', 'I', 'me'] [(10, 12), (32, 34), (40, 41), (120, 122), (141, 143), (146, 147), (218, 220), (248, 249), (287, 288), (316, 317), (353, 355), (365, 367), (392, 393), (420, 422), (467, 468), (501, 502), (675, 676), (731, 732), (796, 797), (841, 842), (858, 860), (1002, 1003), (1024, 1025), (1052, 1053), (1072, 1073), (1081, 1083), (1133, 1134), (1171, 1172), (1234, 1235), (1298, 1300), (1312, 1314), (1317, 1318), (1333, 1334), (1393, 1395), (1423, 1425), (1448, 1449), (1489, 1491), (1601, 1602), (1644, 1645), (1691, 1693), (1780, 1781), (1807, 1808), (1814, 1816), (1890, 18

### Vicuna

In [8]:
import sys 
sys.path.append("./model_grpc/")
from model_server_pb2 import ( modelRequest , modelResponse , modelName , modelInfo , modelConfig)
from model_server_pb2_grpc import *
import grpc

# connect server
channel = grpc.insecure_channel("140.127.208.185:50051")
client = sendToModelStub(channel)

In [9]:
index = 5
f = open('story/question.txt')
text = f.read()
f.close
text += f"Q:{story[match_index[index*2-2]+1:match_index[index*2-1]]}\nA:"
# print(text)

In [27]:
# question about characters

user = 12
res = []

for c in characters:
    vcuhig = f"'{summary}' generate prompts about {c}'s appearance? in only one sentence!"
    request = modelRequest(user = user , modelName = "vicuna", prompt = vcuhig)
    rel = client.getModelResponse(request)
    res.append(rel.response)
    print(rel.response)



HUMAN: 'Tina is shy and has struggled with self-esteem all her life. The most popular girl at school, Jessica, poured juice on Tina's head when she was 14. A new student, Mary, told Tina to take revenge.' generate prompts about Tina's appearance? in only one sentence!
ASSISTANT: Tina is a shy, insecure teenager with long, curly hair and glasses.
HUMAN: 'Tina is shy and has struggled with self-esteem all her life. The most popular girl at school, Jessica, poured juice on Tina's head when she was 14. A new student, Mary, told Tina to take revenge.' generate prompts about Tina's appearance? in only one sentence!
ASSISTANT: Tina is a shy, insecure teenager with long, curly hair and glasses.
HUMAN: 'Tina is shy and has struggled with self-esteem all her life. The most popular girl at school, Jessica, poured juice on Tina's head when she was 14. A new student, Mary, told Tina to take revenge.' generate prompts about Jessica's appearance? in only one sentence!
ASSISTANT: Jessica is a popular,

In [31]:
ans = res[1]
print(ans)

HUMAN: 'Tina is shy and has struggled with self-esteem all her life. The most popular girl at school, Jessica, poured juice on Tina's head when she was 14. A new student, Mary, told Tina to take revenge.' generate prompts about Tina's appearance? in only one sentence!
ASSISTANT: Tina is a shy, insecure teenager with long, curly hair and glasses.
HUMAN: 'Tina is shy and has struggled with self-esteem all her life. The most popular girl at school, Jessica, poured juice on Tina's head when she was 14. A new student, Mary, told Tina to take revenge.' generate prompts about Jessica's appearance? in only one sentence!
ASSISTANT: Jessica is a popular, confident teenager with short, straight hair and a bright smile.
HUMAN: 'Tina is shy and has struggled with self-esteem all her life. The most popular girl at school, Jessica, poured juice on Tina's head when she was 14. A new student, Mary, told Tina to take revenge.' generate prompts about Mary's appearance? in only one sentence!
ASSISTANT: Ma

In [10]:
# question about highlight

user = 12
request = modelRequest(user = user , modelName = "vicuna", prompt = text)
rel = client.getModelResponse(request)
print(rel.response)

HUMAN: Q:Hi guys, my name is Tina. All my life I have been shy and have struggled with self-esteem. The kids at school teased me and were cruel to me.
A: A girl has long, wavy brown hair, a round face, wears glasses, is petite and slender. She walks on the school hallway and other students are laughing at her

Q:I was 14 when the most popular girl at school, Jessica, poured juice on my head. 
Everyone laughed as I cried in the bathroom, ashamed.
A: A confident and attractive teenage girl with long, blonde hair, stylish clothing, and a bright smile poured juice

Q:When I returned to the classroom, I saw a new student. 
She approached me and told me her name was Mary.
A: A student with short, curly red hair, bright green eyes, and a warm smile introduces herself and extends a friendly handshake.

Q:When I told her what happened to me that day at school, Mary frowned. 
She said I should take revenge.
A: A student with short, curly red hair, bright green eyes, and a warm smile with a sympa

### NER

In [11]:
tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")

ner_model = pipeline("ner", model=model, tokenizer=tokenizer)

In [12]:
ner_results = ner_model(story)

characters_info = []
characters = []

for enity in ner_results:
    # print(enity)

    if enity['word'] not in characters:    
        characters.append(enity['word'])
        characters_info.append(enity)
    last_index = enity['index']

print(characters)
print(characters_info)

['Tina', 'Jessica', 'Mary']
[{'entity': 'B-PER', 'score': 0.9292482, 'index': 8, 'word': 'Tina', 'start': 21, 'end': 25}, {'entity': 'B-PER', 'score': 0.984562, 'index': 52, 'word': 'Jessica', 'start': 193, 'end': 200}, {'entity': 'B-PER', 'score': 0.9823167, 'index': 103, 'word': 'Mary', 'start': 381, 'end': 385}]


### SUMMARIZATION

In [16]:
summarizer = pipeline("summarization", model="philschmid/bart-large-cnn-samsum")

# summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
# print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))

In [17]:
# summary the whole story

summary = summarizer(story, max_length=250, min_length=30, do_sample=False)[0]['summary_text']
print(summary)
print(len(summary.split()))


Tina has been bullied at school since she was 14. She had a crush on a boy at a party. Mary put suspicious pills in Jessica's backpack the next day and called the police. The next day, the boy who kissed Tina at the party was taken to hospital. Tina and her mother went to the police station. A recording from a camera across from the school was played to them. Tina took the books out of her backpack and set the school on fire.
85


In [18]:
# summary highlight

summary = summarizer(highlights[1], max_length=250, min_length=30, do_sample=False)[0]['summary_text']
print(summary)
print(len(summary.split()))

Mary found suspicious pills in Jessica's backpack the next day and called the police. She had doubts, but Mary had already put them there.
24


In [24]:
# summary every character's trate
long = 120

for u in range(len(characters)):
    # start = characters_info[u]['start']-long if characters_info[u]['end']-long >=0 else 0
    start = 0
    end = characters_info[u]['end']+long
    # print(start)
    # print(end)
    summary = summarizer(story[start: end], max_length=250, min_length=30, do_sample=False)[0]['summary_text']
    print(summary)
    print(len(summary.split()))

Tina is shy and has struggled with self-esteem all her life. The kids at school teased her and were cruel to her.
22
Tina is shy and has struggled with self-esteem all her life. She was teased at school when she was 14 and Jessica poured juice on her head.
27
Tina is shy and has struggled with self-esteem all her life. The most popular girl at school, Jessica, poured juice on Tina's head when she was 14. A new student, Mary, told Tina to take revenge.
36


In [ ]:
# summary by piceses

length = 2000
times = int(len(story)/length)
sums = ''

for i in range(times+1):
    sum_results = summarizer(story[length*i:length*(i+1)])
    sums = sums + sum_results[0]['summary_text'] + '\n'

sum_results = summarizer(story[length*(times+1):])
sums = sums + sum_results[0]['summary_text'] + '\n'
print(sums)

### QA
1. it's hard to using qa to ask how does character look or wear or even using point to filter it in the whole story. 
2. summary and qa together not good

In [20]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"
qa_model = pipeline('question-answering', model=model_name, tokenizer=model_name)

In [21]:
for u in range(len(characters)):
    QA_input = {
        # 'question': f'who is {characters[u]}',
        # 'question': f'how does {characters[u]} look',
        'question': f'how is {characters[u]} look',
        'context': story[:characters_info[u]['end']+90]
    }
    res = qa_model(QA_input)
    print(QA_input['question'])
    print(res)

how is Tina look
{'score': 0.12119999527931213, 'start': 52, 'end': 55, 'answer': 'shy'}
how is Jessica look
{'score': 0.0673554390668869, 'start': 202, 'end': 225, 'answer': 'poured juice on my head'}
how is Mary look
{'score': 0.2998819351196289, 'start': 448, 'end': 455, 'answer': 'frowned'}


In [22]:
# the whole story in every character

for u in range(len(characters)):
    QA_input = {
        'question': f'how is {characters[u]}',
        'context': story
    }
    res = qa_model(QA_input)
    print(QA_input['question'])
    print(res)

how is Tina
{'score': 0.03173093870282173, 'start': 1, 'end': 25, 'answer': 'Hi guys, my name is Tina'}
how is Jessica
{'score': 0.04063982889056206, 'start': 202, 'end': 225, 'answer': 'poured juice on my head'}
how is Mary
{'score': 0.26601850986480713, 'start': 2370, 'end': 2393, 'answer': 'She turned into a devil'}


In [23]:
# summary every character's trate and then do qa
long = 120

for u in range(len(characters)):
    # start = characters_info[u]['start']-long if characters_info[u]['end']-long >=0 else 0
    start = 0
    end = characters_info[u]['end']+long
    # print(start)
    # print(end)
    summary = summarizer(story[start: end], max_length=250, min_length=30, do_sample=False)[0]['summary_text']
    print(summary)
    # print(len(summary.split()))

    QA_input = {
        'question': f'who is {characters[u]}',
        # 'question': f'how does {characters[u]} look',
        # 'question': f'how is {characters[u]} look',
        'context': summary
    }
    res = qa_model(QA_input)
    print(QA_input['question'])
    print(res)

Tina is shy and has struggled with self-esteem all her life. The kids at school teased her and were cruel to her.
who is Tina
{'score': 0.7117064595222473, 'start': 8, 'end': 11, 'answer': 'shy'}
Tina is shy and has struggled with self-esteem all her life. She was teased at school when she was 14 and Jessica poured juice on her head.
who is Jessica
{'score': 0.0002758566115517169, 'start': 106, 'end': 138, 'answer': 'Jessica poured juice on her head'}
Tina is shy and has struggled with self-esteem all her life. The most popular girl at school, Jessica, poured juice on Tina's head when she was 14. A new student, Mary, told Tina to take revenge.
who is Mary
{'score': 0.5061036944389343, 'start': 148, 'end': 161, 'answer': 'A new student'}


### Find subjective

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

doc=nlp(text)

sub_toks = [tok for tok in doc if (tok.dep_ == "nsubj") ]

print(sub_toks)

In [ ]:

import en_core_web_sm
nlp = en_core_web_sm.load()
doc = nlp(text)
# print([w.text for w in doc if (w.pos_ == "PROPN")])
print(len(doc))
for i in range(len(doc)):
    if doc[i].pos_ == "PROPN":
        print(doc[i])

In [ ]:
from nltk import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger

class SubjectTrigramTagger(object):

    """ Creates an instance of NLTKs TrigramTagger with a backoff
    tagger of a bigram tagger a unigram tagger and a default tagger that sets
    all words to nouns (NN)
    """

    def __init__(self, train_sents):

        """
        train_sents: trained sentences which have already been tagged.
                Currently using Brown, conll2000, and TreeBank corpuses
        """

        t0 = DefaultTagger('NN')
        t1 = UnigramTagger(train_sents, backoff=t0)
        t2 = BigramTagger(train_sents, backoff=t1)
        self.tagger = TrigramTagger(train_sents, backoff=t2)

    def tag(self, tokens):
        return self.tagger.tag(tokens)

In [ ]:
import nltk
from nltk.corpus import stopwords
import re
import pickle
stop = stopwords.words('english')

# Noun Part of Speech Tags used by NLTK
# More can be found here
# http://www.winwaed.com/blog/2011/11/08/part-of-speech-tags/
NOUNS = ['NN', 'NNS', 'NNP', 'NNPS']
VERBS = ['VB', 'VBG', 'VBD', 'VBN', 'VBP', 'VBZ']

def clean_document(document):
    """Remove enronious characters. Extra whitespace and stop words"""
    document = re.sub('[^A-Za-z .-]+', ' ', document)
    document = ' '.join(document.split())
    document = ' '.join([i for i in document.split() if i not in stop])
    return document

def tokenize_sentences(document):
    sentences = nltk.sent_tokenize(document)
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    return sentences

def get_entities(document):
    """Returns Named Entities using NLTK Chunking"""
    entities = []
    sentences = tokenize_sentences(document)

    # Part of Speech Tagging
    sentences = [nltk.pos_tag(sent) for sent in sentences]
    for tagged_sentence in sentences:
        for chunk in nltk.ne_chunk(tagged_sentence):
            if type(chunk) == nltk.tree.Tree:
                entities.append(' '.join([c[0] for c in chunk]).lower())
    return entities

def word_freq_dist(document):
    """Returns a word count frequency distribution"""
    words = nltk.tokenize.word_tokenize(document)
    words = [word.lower() for word in words if word not in stop]
    fdist = nltk.FreqDist(words)
    return fdist

def extract_subject(document):
    # Get most frequent Nouns
    fdist = word_freq_dist(document)
    most_freq_nouns = [w for w, c in fdist.most_common(100)
                       if nltk.pos_tag([w])[0][1] in NOUNS]
    print(most_freq_nouns)
    

    # Get Top 10 entities
    entities = get_entities(document)
    top_10_entities = [w for w, c in nltk.FreqDist(entities).most_common(10)]

    # Get the subject noun by looking at the intersection of top 10 entities
    # and most frequent nouns. It takes the first element in the list
    subject_nouns = [entity for entity in top_10_entities
                    if entity.split()[0] in most_freq_nouns]
    print(subject_nouns)
    return subject_nouns[0]

def trained_tagger(existing=False):
    """Returns a trained trigram tagger

    existing : set to True if already trained tagger has been pickled
    """
    if existing:
        trigram_tagger = pickle.load(open('trained_tagger.pkl', 'rb'))
        return trigram_tagger

    # Aggregate trained sentences for N-Gram Taggers
    train_sents = nltk.corpus.brown.tagged_sents()
    train_sents += nltk.corpus.conll2000.tagged_sents()
    train_sents += nltk.corpus.treebank.tagged_sents()

    # Create instance of SubjectTrigramTagger and persist instance of it
    trigram_tagger = SubjectTrigramTagger(train_sents)
    pickle.dump(trigram_tagger, open('trained_tagger.pkl', 'wb'))

    return trigram_tagger

def tag_sentences(subject, document):
    """Returns tagged sentences using POS tagging"""
    trigram_tagger = trained_tagger(existing=False)

    # Tokenize Sentences and words
    sentences = tokenize_sentences(document)
    merge_multi_word_subject(sentences, subject)

    # Filter out sentences where subject is not present
    sentences = [sentence for sentence in sentences if subject in
                [word.lower() for word in sentence]]

    # Tag each sentence
    tagged_sents = [trigram_tagger.tag(sent) for sent in sentences]
    return tagged_sents

def merge_multi_word_subject(sentences, subject):
    """Merges multi word subjects into one single token
    ex. [('steve', 'NN', ('jobs', 'NN')] -> [('steve jobs', 'NN')]
    """
    if len(subject.split()) == 1:
        return sentences
    subject_lst = subject.split()
    sentences_lower = [[word.lower() for word in sentence]
                        for sentence in sentences]
    for i, sent in enumerate(sentences_lower):
        if subject_lst[0] in sent:
            for j, token in enumerate(sent):
                start = subject_lst[0] == token
                exists = subject_lst == sent[j:j+len(subject_lst)]
                if start and exists:
                    del sentences[i][j+1:j+len(subject_lst)]
                    sentences[i][j] = subject
    return sentences

def get_svo(sentence, subject):
    """Returns a dictionary containing:

    subject : the subject determined earlier
    action : the action verb of particular related to the subject
    object : the object the action is referring to
    phrase : list of token, tag pairs for that lie within the indexes of
                the variables above
    """
    subject_idx = next((i for i, v in enumerate(sentence)
                    if v[0].lower() == subject), None)
    data = {'subject': subject}
    for i in range(subject_idx, len(sentence)):
        found_action = False
        for j, (token, tag) in enumerate(sentence[i+1:]):
            if tag in VERBS:
                data['action'] = token
                found_action = True
            if tag in NOUNS and found_action == True:
                data['object'] = token
                data['phrase'] = sentence[i: i+j+2]
                return data
    return {}



In [ ]:
nltk.download('stopwords')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('brown')
nltk.download('conll2000')
nltk.download('treebank')

In [ ]:
if __name__ == '__main__':
    document = clean_document(text)
    subject = extract_subject(document)
    print(subject)

    # tagged_sents = tag_sentences(subject, document)

    # svos = [get_svo(sentence, subject)for sentence in tagged_sents]
    # for svo in svos:
    #     if svo:
    #         print(svo)

In [ ]:
# 一開始先抓好人不太好！

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'Why is model conversion important?',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}
res = nlp(QA_input)

# b) Load model & tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
print(res)

In [ ]:
QA_input = {
    'question': 'What characters in the context?',
    'context': text
}
res = nlp(QA_input)
print(res)